# Objectif du Notebook
Dans ce notebook, notre objectif principal est de fusionner les différentes tables liées aux produits, qui sont stockées dans notre Data Warehouse. Cette fusion permettra de regrouper toutes les informations pertinentes sur les produits en un seul jeu de données cohérent et consolidé.

En effet, ces tables contiennent des informations clés, telles que les catégories de produits, les sous-catégories, les régions de vente, ou encore les détails des commandes. En combinant ces données, nous simplifions leur structure pour qu'elles soient prêtes à être utilisées dans des algorithmes de machine learning. Cela facilitera des tâches telles que la prédiction des ventes, l'analyse des tendances, ou encore le regroupement des produits en fonction de leur performance.

Ce processus de fusion est une étape essentielle dans le pipeline de préparation des données, car il garantit que toutes les relations entre les différentes entités du Data Warehouse sont prises en compte et intégrées dans le jeu de données final.

# Connexion à la base de données

Cette section configure et établit une connexion à la base de données SQL Server à l'aide de pyodbc.

In [3]:
import pyodbc
import pandas as pd

# Configuration de la chaîne de connexion
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"  # Remplacez par votre pilote installé
    "Server=DESKTOP-1RQBJ5I;"  # Remplacez par le nom de votre serveur
    "Database=AVW_DW;"  # Nom de votre base de données
    "Trusted_Connection=yes;"  # Utilisation de l'authentification Windows
)

# Define a function to fetch data from SQL database for all tables
def fetch_all_tables_from_sql():
    try:
        # Établir une connexion à la base de données
        with pyodbc.connect(conn_str) as conn:
            print("Connexion réussie à la base de données AVW_DW !")
            
            # Fetch data for each table into pandas DataFrames
            dim_year = pd.read_sql("SELECT * FROM Dim_Year", conn)
            dim_month = pd.read_sql("SELECT * FROM Dim_Month", conn)
            dim_date = pd.read_sql("SELECT * FROM Dim_Date", conn)
            dim_salesperson = pd.read_sql("SELECT * FROM Dim_SalesPerson", conn)
            dim_territory = pd.read_sql("SELECT * FROM Dim_Territory", conn)
            dim_product = pd.read_sql("SELECT * FROM Dim_Product", conn)
            dim_productsubcategory = pd.read_sql("SELECT * FROM Dim_ProductSubCategory", conn)
            dim_productcategory = pd.read_sql("SELECT * FROM Dim_ProductCategory", conn)
            fact_salesorder = pd.read_sql("SELECT * FROM Fact_SalesOrder", conn)
            fact_product = pd.read_sql("SELECT * FROM Fact_Product", conn)
        
            # Return all DataFrames as a dictionary for easy access
            return {
                "Dim_Year": dim_year,
                "Dim_Month": dim_month,
                "Dim_Date": dim_date,
                "Dim_SalesPerson": dim_salesperson,
                "Dim_Territory": dim_territory,
                "Dim_Product": dim_product,
                "Dim_ProductSubCategory": dim_productsubcategory,
                "Dim_ProductCategory": dim_productcategory,
                "Fact_SalesOrder": fact_salesorder,
                "Fact_Product": fact_product,
            }
    except Exception as e:
        print("Erreur lors de la connexion ou de la récupération des données :", e)
        return {}

# Fetch all tables
tables = fetch_all_tables_from_sql()

# Example: Print the first few rows of Dim_Year and Fact_SalesOrder tables
if tables:  # Ensure tables are loaded before printing
    print(tables["Dim_Year"].head())
    print(tables["Fact_SalesOrder"].head())




Connexion réussie à la base de données AVW_DW !
  YearKey  Year  IsDeleted
0    2011  2011      False
1    2012  2012      False
2    2013  2013      False
     Id   DateKey  TerritoryKey  SalePersonKey   Revenue  NumberOrder  \
0  7393  01102011             5              1   2417.47            1   
1  7394  30102012             5              1  90167.33            1   
2  7395  30082012             4              1  33206.02            1   
3  7396  30082012             2              1  29638.27            1   
4  7397  30062013             6              1  17990.96            1   

   IsDeleted  
0      False  
1      False  
2      False  
3      False  
4      False  


C:\Users\PE\AppData\Local\Temp\ipykernel_38316\1385756148.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_year = pd.read_sql("SELECT * FROM Dim_Year", conn)
C:\Users\PE\AppData\Local\Temp\ipykernel_38316\1385756148.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_month = pd.read_sql("SELECT * FROM Dim_Month", conn)
C:\Users\PE\AppData\Local\Temp\ipykernel_38316\1385756148.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_date = pd.read_sql("SELECT * FROM Dim_Date", conn)
C:\Users\PE\AppData\Local\Tem

# Fusion des tables

Cette section fusionne les différentes tables à l'aide des clés de jointure identifiées

In [17]:
# Afficher les colonnes des DataFrames pour vérifier les clés de jointure disponibles
print("Colonnes de Fact_SalesOrder:", tables["Fact_SalesOrder"].columns)
print("Colonnes de Dim_Product:", tables["Dim_Product"].columns)

# Utilisez les clés correctes pour la jointure
merged_df = pd.merge(tables["Fact_SalesOrder"], tables["Dim_Date"], on='DateKey', how='left')
merged_df = pd.merge(merged_df, tables["Dim_Territory"], on='TerritoryKey', how='left')

# Vérifiez si 'ProductKey' est la bonne clé pour Dim_Product, sinon utilisez la clé correcte
if 'ProductKey' in tables["Dim_Product"].columns and 'ProductKey' in tables["Fact_SalesOrder"].columns:
    merged_df = pd.merge(merged_df, tables["Dim_Product"], on='ProductKey', how='left')
else:
    print("La clé 'ProductKey' n'est pas disponible dans les tables fournies.")
    # Identifiez et utilisez la bonne clé de jointure ici

# Continuez avec les autres jointures


Colonnes de Fact_SalesOrder: Index(['Id', 'DateKey', 'TerritoryKey', 'SalePersonKey', 'Revenue',
       'NumberOrder'],
      dtype='object')
Colonnes de Dim_Product: Index(['ProductKey', 'Name', 'ProductNumber', 'StandardCost', 'ListPrice',
       'Weight', 'ProductSubCategoryKey', 'ProductId'],
      dtype='object')
La clé 'ProductKey' n'est pas disponible dans les tables fournies.


# Vérification des colonnes des tables

Cette section affiche les colonnes disponibles dans chaque table pour identifier les clés de jointure.

In [20]:
# Afficher les colonnes de chaque table
print("Colonnes de Fact_SalesOrder:", tables["Fact_SalesOrder"].columns)
print("Colonnes de Dim_Product:", tables["Dim_Product"].columns)
print("Colonnes de Dim_Date:", tables["Dim_Date"].columns)
print("Colonnes de Dim_Territory:", tables["Dim_Territory"].columns)


Colonnes de Fact_SalesOrder: Index(['Id', 'DateKey', 'TerritoryKey', 'SalePersonKey', 'Revenue',
       'NumberOrder'],
      dtype='object')
Colonnes de Dim_Product: Index(['ProductKey', 'Name', 'ProductNumber', 'StandardCost', 'ListPrice',
       'Weight', 'ProductSubCategoryKey', 'ProductId'],
      dtype='object')
Colonnes de Dim_Date: Index(['DateKey', 'MonthKey', 'DATE'], dtype='object')
Colonnes de Dim_Territory: Index(['TerritoryKey', 'TerritoryId', 'Name', 'ContryRegionCode'], dtype='object')


In [8]:
# Supprimer la colonne 'IsDeleted' dans toutes les tables si elle existe
for table_name, table_data in tables.items():
    if "IsDeleted" in table_data.columns:
        tables[table_name].drop(columns=["IsDeleted"], inplace=True)

In [26]:
# Merge Dim_ProductSubCategory first to ensure ProductCategoryKey is included in merged_df
merged_df = pd.merge(merged_df, tables["Dim_ProductSubCategory"], on='ProductSubCategoryKey', how='left')

# Verify that ProductCategoryKey is now present
print(merged_df.columns)


Index(['Id', 'DateKey', 'TerritoryKey', 'ProductKey', 'Qty', 'TerritoryId',
       'Name_x', 'ContryRegionCode', 'MonthKey', 'DATE', 'ProductNumber',
       'StandardCost', 'ListPrice', 'Weight', 'ProductSubCategoryKey',
       'ProductId', 'ProductSubCategoryId', 'Name', 'ProductCategoryKey'],
      dtype='object')


In [27]:
# Now merge Dim_ProductCategory with the existing merged_df
merged_df = pd.merge(merged_df, tables["Dim_ProductCategory"], on='ProductCategoryKey', how='left', suffixes=('_merged', '_category'))

# Check the final columns
print(merged_df.columns)


Index(['Id', 'DateKey', 'TerritoryKey', 'ProductKey', 'Qty', 'TerritoryId',
       'Name_x', 'ContryRegionCode', 'MonthKey', 'DATE', 'ProductNumber',
       'StandardCost', 'ListPrice', 'Weight', 'ProductSubCategoryKey',
       'ProductId', 'ProductSubCategoryId', 'Name_merged',
       'ProductCategoryKey', 'ProductCategoryId', 'Name_category'],
      dtype='object')


# Exportation des données

Cette section exporte le DataFrame final fusionné dans un fichier Excel.

In [28]:
merged_df.to_excel(r"C:\Users\PE\Desktop\tesdt.xlsx", index=False)

: 